# Prerequisites Library

OIDv4 Dataset Toolkit Downloader Github Repository [Link](https://github.com/EscVM/OIDv4_ToolKit) and Documentation



In [31]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

fatal: destination path 'OIDv4_ToolKit' already exists and is not an empty directory.


In [32]:
!pip3 install -r /content/OIDv4_ToolKit/requirements.txt

In [44]:
import os
from pprint import pprint
from six import BytesIO
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
from PIL import Image, ImageColor, ImageDraw, ImageFont, ImageOps

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import models
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

# Load the Dataset
  Dataset contains 5 classes : 
*   Ambulance
*   Bus
*   Car
*   Truck
*   Van

## Train Dataset

In [34]:
!python /content/OIDv4_ToolKit/main.py downloader --classes Ambulance Truck Van Bus Car --type_csv train --limit 500


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Ambulance.
   [ERROR] | Missing the train-annotations-bbox.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...100%, 1138 MB, 35413 KB/s, 32 seconds passed
[DOWNLOAD] | Fil

## Test Dataset

In [33]:
!python /content/OIDv4_ToolKit/main.py downloader --classes Ambulance Truck Van Bus Car --type_csv test --limit 500


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Ambulance.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 27147 KB/s, 0 seconds passed
[DOWNLOAD] | Fil

## Validation Dataset

In [58]:
!python /content/OIDv4_ToolKit/main.py downloader --classes Ambulance Truck Van Bus Car --type_csv validation --limit 500


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Ambulance.
   [ERROR] | Missing the validation-annotations-bbox.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...100%, 16 MB, 46147 KB/s, 0 seconds passed
[DOWNLOAD] | F

# Dataset Preprocessing

### Normalize and Augmentation


In [59]:
# Training Dataset Normalize and Augmentation
train_datagen=ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=40,
    horizontal_flip=True,
    fill_mode='nearest'
    )

test_datagen=ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
    )

validation_datagen=ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
    )

traindir = "/content/OID/Dataset/train" 

train_generator=train_datagen.flow_from_directory(
    traindir,
    target_size =(224,224),
    class_mode="categorical",
    batch_size=64
    )

testdir = "/content/OID/Dataset/test"

test_generator=test_datagen.flow_from_directory(
    testdir,
    target_size =(224,224),
    class_mode="categorical",
    batch_size=64
    )

valdir = "/content/OID/Dataset/validation"

validation_generator=validation_datagen.flow_from_directory(
    valdir,
    target_size =(224,224),
    class_mode="categorical",
    batch_size=64
    )

class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes),
    train_generator.classes
    )

print(train_generator.labels)
print(train_generator.class_indices)
print(class_weights)

Found 2338 images belonging to 5 classes.
Found 1718 images belonging to 5 classes.
Found 995 images belonging to 5 classes.
[0 0 0 ... 4 4 4]
{'Ambulance': 0, 'Bus': 1, 'Car': 2, 'Truck': 3, 'Van': 4}
[1.38343195 0.9352     0.9352     0.9352     0.9352    ]


# Model Building

## coba model mobilenet v2

In [ ]:
def format_prediction_string(testdir, result):
    prediction_strings = []
    
    for i in range(len(result['detection_scores'])):
        class_name = result['detection_class_names'][i].decode("utf-8")
        YMin,XMin,YMax,XMax = result['detection_boxes'][i]
        score = result['detection_scores'][i]
        
        prediction_strings.append(
            f"{class_name} {score} {XMin} {YMin} {XMax} {YMax}"
        )
        
    prediction_string = " ".join(prediction_strings)

    return {
        "ImageID": testdir,
        "PredictionString": prediction_string
    }

In [ ]:
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.axis('off')
    plt.imshow(image)

In [ ]:
 def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype(
            "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
            25)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()
    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i].tolist())
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                           int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
                image_pil,
                ymin,
                xmin,
                ymax,
                xmax,
                color,
                font,
                display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

In [ ]:
sample_image_path = "../input/test/6beb79b52308112d.jpg"

with tf.Graph().as_default():
    # Create our inference graph
    image_string_placeholder = tf.placeholder(tf.string)
    decoded_image = tf.image.decode_jpeg(image_string_placeholder)
    decoded_image_float = tf.image.convert_image_dtype(
        image=decoded_image, dtype=tf.float32
    )
    # Expanding image from (height, width, 3) to (1, height, width, 3)
    image_tensor = tf.expand_dims(decoded_image_float, 0)

    # Load the model from tfhub.dev, and create a detector_output tensor
    model_url = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
    detector = hub.Module(model_url)
    detector_output = detector(image_tensor, as_dict=True)
    # Initialize the Session
    init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]
    sess = tf.Session()
    sess.run(init_ops)

    # Load our sample image into a binary string
    with tf.gfile.Open(sample_image_path, "rb") as binfile:
        image_string = binfile.read()

    # Run the graph we just created
    result_out, image_out = sess.run(
        [detector_output, decoded_image],
        feed_dict={image_string_placeholder: image_string}
    )

AttributeError: ignored

## Coba Transfer Learning
ImageNet V2

### Create Base Model

In [54]:
IMG_SIZE = (224,224)
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


### Feature Extraction

In [64]:
image_batch, label_batch = next(iter(train_generator))

feature_batch = base_model(image_batch)

#Freeze the convolutional layers
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(5)
prediction_batch = prediction_layer(feature_batch_average)

#Model Building
inputs = tf.keras.Input(shape=( 224, 224, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

print(feature_batch.shape)
print(feature_batch_average.shape)
print(prediction_batch.shape)

(64, 7, 7, 1280)
(64, 1280)
(64, 5)


In [66]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [68]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [69]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_generator)

16/16 [==============================] - 54s 3s/step - loss: 1.6228 - accuracy: 0.4354


### Re-Train the Model

In [70]:
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=validation_generator)

Epoch 1/10
37/37 [==============================] - 200s 5s/step - loss: 1.8789 - accuracy: 0.2023 - val_loss: 1.5142 - val_accuracy: 0.3739
Epoch 2/10
37/37 [==============================] - 179s 5s/step - loss: 1.7148 - accuracy: 0.2070 - val_loss: 1.5312 - val_accuracy: 0.3005
Epoch 3/10
37/37 [==============================] - 179s 5s/step - loss: 1.7099 - accuracy: 0.2096 - val_loss: 1.5500 - val_accuracy: 0.2985
Epoch 4/10
37/37 [==============================] - 179s 5s/step - loss: 1.7020 - accuracy: 0.2092 - val_loss: 1.5595 - val_accuracy: 0.2643
Epoch 5/10
37/37 [==============================] - 187s 5s/step - loss: 1.6738 - accuracy: 0.2271 - val_loss: 1.5683 - val_accuracy: 0.2372
Epoch 6/10
37/37 [==============================] - 178s 5s/step - loss: 1.6790 - accuracy: 0.2177 - val_loss: 1.5649 - val_accuracy: 0.2965
Epoch 7/10
37/37 [==============================] - 178s 5s/step - loss: 1.6766 - accuracy: 0.2117 - val_loss: 1.5725 - val_accuracy: 0.2181
Epoch 8/10
37

### Fine Tuning

In [73]:
#Un-Freeze Top Layer
base_model.trainable = True

# Total Layer on Based Model
print("Total Layer on Based Model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Total Layer on Based Model:  154


In [74]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [75]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 1,867,845
Non-trainable params: 396,544
_________________________________________________________________


In [77]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_generator)

Epoch 10/20
37/37 [==============================] - 243s 6s/step - loss: 1.6636 - accuracy: 0.2246 - val_loss: 1.6960 - val_accuracy: 0.1186
Epoch 11/20
37/37 [==============================] - 234s 6s/step - loss: 1.6098 - accuracy: 0.2092 - val_loss: 1.5419 - val_accuracy: 0.1548
Epoch 12/20
37/37 [==============================] - 232s 6s/step - loss: 1.5942 - accuracy: 0.2083 - val_loss: 1.5793 - val_accuracy: 0.0734
Epoch 13/20
37/37 [==============================] - 231s 6s/step - loss: 1.5940 - accuracy: 0.2100 - val_loss: 1.5717 - val_accuracy: 0.2804
Epoch 14/20
37/37 [==============================] - 232s 6s/step - loss: 1.5699 - accuracy: 0.2641 - val_loss: 1.5080 - val_accuracy: 0.4060
Epoch 15/20
37/37 [==============================] - 233s 6s/step - loss: 1.5729 - accuracy: 0.2436 - val_loss: 1.6438 - val_accuracy: 0.1608
Epoch 16/20
37/37 [==============================] - 232s 6s/step - loss: 1.5757 - accuracy: 0.2419 - val_loss: 1.6919 - val_accuracy: 0.1095
Epoch 